In [186]:
import pandas as pd
import numpy as np
import html5lib
from bs4 import BeautifulSoup
import requests 

In [187]:
c = pd.read_csv('worldcities.csv')

In [188]:
c['capital'] = c['capital'].fillna('')
c['admin_name'] = c['admin_name'].fillna('')
c['population'] = c['population'].fillna(0)
c['population'] = c['population'].astype('int32')
c = c.set_index('id')
c = c.drop(columns=["iso2", "capital", "city"])
c

,city_ascii,lat,lng,country,iso3,admin_name,population
id,,,,,,,
1392685764,Tokyo,35.6897,139.6922,Japan,JPN,Tōkyō,37732000
1360771077,Jakarta,-6.1750,106.8275,Indonesia,IDN,Jakarta,33756000
1356872604,Delhi,28.6100,77.2300,India,IND,Delhi,32226000
1156237133,Guangzhou,23.1300,113.2600,China,CHN,Guangdong,26940000
1356226629,Mumbai,19.0761,72.8775,India,IND,Mahārāshtra,24973000
...,...,...,...,...,...,...,...
1643985006,Numto,63.6667,71.3333,Russia,RUS,Khanty-Mansiyskiy Avtonomnyy Okrug-Yugra,10
1304217709,Nord,81.7166,-17.8000,Greenland,GRL,,10
1304206491,Timmiarmiut,62.5333,-42.2167,Greenland,GRL,Kujalleq,10


In [189]:
c = c[c['population'] > 200000]

In [190]:
# HTML PARSE FOR REGIONS

In [191]:
url = 'https://cloford.com/resources/codes/index.htm'
r = requests.get(url) 
soup = BeautifulSoup(r.content, 'html5lib')

In [192]:
table = pd.read_html(str(soup.find('table', {'class': 'outlinetable'})))
regions = table[0] #parses all tables and returns list, but we only have one.

In [193]:
cols = regions[['Continent', 'Region', 'ISO (3)']]
cols = cols.rename(columns={"ISO (3)": 'iso3'})
cols = cols.set_index('iso3')

In [194]:
cols.loc['JPN', 'Continent']

'Asia'

In [195]:
c['continent'] = np.zeros(len(c))
c['region'] = np.zeros(len(c))

for i, r in c.iterrows():
    if r['iso3'] in cols.index:
        c.loc[i, 'continent'] = cols.loc[r['iso3'], 'Continent']
        c.loc[i, 'region'] = cols.loc[r['iso3'], 'Region']
    else:
        r['continent'] = ''
        r['region'] = ''

c

C:\Users\BrandonKaminski\AppData\Local\Temp\ipykernel_32352\1436035958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c['continent'] = np.zeros(len(c))
C:\Users\BrandonKaminski\AppData\Local\Temp\ipykernel_32352\1436035958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c['region'] = np.zeros(len(c))


,city_ascii,lat,lng,country,iso3,admin_name,population,continent,region
id,,,,,,,,,
1392685764,Tokyo,35.6897,139.6922,Japan,JPN,Tōkyō,37732000,Asia,East Asia
1360771077,Jakarta,-6.1750,106.8275,Indonesia,IDN,Jakarta,33756000,Asia,South East Asia
1356872604,Delhi,28.6100,77.2300,India,IND,Delhi,32226000,Asia,South Asia
1156237133,Guangzhou,23.1300,113.2600,China,CHN,Guangdong,26940000,Asia,East Asia
1356226629,Mumbai,19.0761,72.8775,India,IND,Mahārāshtra,24973000,Asia,South Asia
...,...,...,...,...,...,...,...,...,...
1356019851,Morena,26.5000,78.0000,India,IND,Madhya Pradesh,200483,Asia,South Asia
1024903782,Soio,-6.1333,12.3667,Angola,AGO,Zaire,200920,Africa,Southern Africa
1862573240,Guacara,10.2333,-67.8833,Venezuela,VEN,Carabobo,200212,Americas,South America


In [196]:
c.to_json('worldcities.json')

In [197]:
c[c['city_ascii'] == 'Philadelphia']

,city_ascii,lat,lng,country,iso3,admin_name,population,continent,region
id,,,,,,,,,
1840000673,Philadelphia,40.0077,-75.1339,United States,USA,Pennsylvania,5512873,Americas,North America
